In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df1 = pd.read_csv("Youtube01-Psy.csv",encoding = "ISO-8859-1")
df2 = pd.read_csv("Youtube02-KatyPerry.csv",encoding = "ISO-8859-1")
df3 = pd.read_csv("Youtube03-LMFAO.csv",encoding = "ISO-8859-1")
df4 = pd.read_csv("Youtube04-Eminem.csv",encoding = "ISO-8859-1")
df5 = pd.read_csv("Youtube05-Shakira.csv",encoding = "ISO-8859-1")

We will train on the df1,df2,df3,df4 and test on df5.

In [3]:
test = df5['CLASS']
df5.drop('CLASS',axis = 1,inplace=True)

In [4]:
df5.shape

(370, 4)

In [5]:
df5.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,Nice songï»¿
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,I love song ï»¿
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,PrÃ¬Ã±Ã§eÅÅ ÃliÅ ÅÃ¸vÃª DÃ¸mÃ­Ã±Ã¸ MÃ¢Äi...,2015-05-28T21:00:08.607000,I love song ï»¿
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,"860,000,000 lets make it first female to reach..."
4,z133stly3kete3tly22petvwdpmghrlli,Analena LÃ³pez,2015-05-28T17:08:29.827000,shakira is best for worldcupï»¿


In [6]:
data = [df1,df2,df3,df4]
final_df = pd.concat(data)

In [7]:
final_df.shape

(1586, 5)

In [8]:
final_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^...,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .ï»¿,1


In [9]:
comment = final_df['CONTENT']

Text files are actually series of words (ordered). In order to run machine learning algorithms we need to convert the text files into numerical feature vectors. We will be using bag of words model for our example. Briefly, we segment each text file into words (for English splitting by space), and count # of times each word occurs in each document and finally assign each word an integer id. Each unique word in our dictionary will correspond to a feature

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(comment)
X_train_counts


<1586x4169 sparse matrix of type '<class 'numpy.int64'>'
	with 20570 stored elements in Compressed Sparse Row format>

Here by doing ‘count_vect.fit_transform(twenty_train.data)’, we are learning the vocabulary dictionary and it returns a Document-Term matrix. [n_samples, n_features].

TF: Just counting the number of words in each document has 1 issue: it will give more weightage to longer documents than shorter documents. To avoid this, we can use frequency (I TF - Term Frequencies) i.e. #count(word) / #Total words, in each document.

TF-IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) which occurs in all document. This is called as TF-IDF i.e Term Frequency times inverse document frequency.


In [11]:
# Below lines of code are used to achieve it.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1586, 4169)

In [25]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='modified_huber', penalty='l2',
                     alpha=1e-3, n_iter=9, random_state=42)),])


In [26]:
target = final_df["CLASS"]
text_clf_svm.fit(comment,target)

D:\miniconda\envs\ml\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

In [27]:
target.shape

(1586,)

In [28]:
predicted_svm = text_clf_svm.predict(df5['CONTENT'])
print(predicted_svm)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0]


In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(test, predicted_svm)

0.9324324324324325

In [30]:
from sklearn.metrics import f1_score
f1_score(test, predicted_svm)

0.9235474006116209